In [100]:
import pandas as pd
import numpy as np
import csv
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
# import seaborn as sns
# import matplotlib.pyplot as plt
from scipy import stats
import sys
from sklearn.impute import SimpleImputer
import random
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [101]:
def read_train():
    train = pd.read_csv("train.csv")

    
    
    train = train[~(train['num_of_cancelled_trips'] >= 6)]
    train = train[~(train['anon_var_1'] >=115)]
    train = train[~(train['anon_var_2'] >= 70)]
    train = train[~(train['anon_var_3'] >= 110 )]
    train = train[~(2>train['customer_score'])]
    train = train[~(train['customer_score']>=3.7 )]
    print(train.columns)
    return train

def hotenc(train):
    mf1 = pd.get_dummies(train['taxi_type'])
    train = train.drop('taxi_type',axis = 1)
    train = train.join(mf1,rsuffix='_Taxitype')

    mf2 = pd.get_dummies(train['customer_score_confidence'])
    train = train.drop('customer_score_confidence',axis = 1)
    train = train.join(mf2,rsuffix='_Customerscore_confidence')

    mf = pd.get_dummies(train['drop_location_type'])
    train = train.drop('drop_location_type',axis = 1)
    train = train.join(mf,rsuffix='_drop_location_type')

    mf = pd.get_dummies(train['sex'])
    train = train.drop('sex',axis = 1)
    train = train.join(mf,rsuffix='sex')
    mf = train['anon_var_1'].isnull()
    mf =mf.astype(int)
    print (mf)
    train = train.join(mf,rsuffix= 'anon_var_1_presence')
    
#     train = train.drop('sex',axis = 1)
#     train = train.drop('distance',axis = 1)
    train = train.drop('anon_var_1',axis = 1)
    train = train.drop('customer_score',axis = 1)
#     train = train.drop('ratings_given_by_cust',axis = 1)
#     train = train.drop('anon_var_3',axis = 1)

    
    
    
    return train

def imputer(data,test):
    
    imp_mean1 = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp_mean2 = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp_median = SimpleImputer(missing_values=np.nan, fill_value = -1)

    
#     dat = np.array(data["customer_score"])
#     imp_mean1.fit(np.resize(dat,(len(dat),1)))
#     dat = imp_mean1.transform(np.resize(dat,(len(dat),1)))
#     data["customer_score"] = dat
# #     print(data.isna().sum())

    
#     dat = np.array(test["customer_score"])
#     dat = imp_mean1.transform(np.resize(dat,(len(dat),1)))
#     test["customer_score"] = dat
#     print(test.isna().sum())

#     dat = np.array(data["anon_var_1"])
#     imp_mean2.fit(np.resize(dat,(len(dat),1)))
#     dat = imp_mean2.transform(np.resize(dat,(len(dat),1)))
#     data["anon_var_1"] = dat
# #     print(data.isna().sum())
    
    
    
#     dat = np.array(test["anon_var_1"])
#     dat = imp_mean1.transform(np.resize(dat,(len(dat),1)))
#     test["anon_var_1"] = dat
    
    
    dat = np.array(data["months_of_activity"])
    imp_median.fit(np.resize(dat,(len(dat),1)))
    dat = imp_median.transform(np.resize(dat,(len(dat),1)))
    data["months_of_activity"] = dat
#     print(data.isna().sum())
    
    
    
    dat = np.array(test["months_of_activity"])
    dat = imp_median.transform(np.resize(dat,(len(dat),1)))
    test["months_of_activity"] = dat


    
#     print(test)
    return data, test

def boxCoxTransform(train, test):
  
    xt, _ = stats.boxcox(train["anon_var_2"])
    xte = stats.boxcox(test["anon_var_2"],lmbda = _)
    train["anon_var_2"] = xt#(xt-xt.mean())/xt.std()
    test["anon_var_2"] = xte#(xte-xt.mean())/xt.std()
    
    xt, _ = stats.boxcox(train["distance"])
    xte = stats.boxcox(test["distance"],lmbda = _)
    train["distance"] = xt #(xt-xt.mean())/xt.std()
    test["distance"] = xte #(xte-xt.mean())/xt.std()
    
#     xt, _ = stats.boxcox(train["customer_score"])
#     xte = stats.boxcox(test["customer_score"],lmbda = _)
#     train["customer_score"] = xt#(xt-xt.mean())/xt.std()
#     test["customer_score"] = xte#(xte-xt.mean())/xt.std()

    xt, _ = stats.boxcox(train["ratings_given_by_cust"])
    xte = stats.boxcox(test["ratings_given_by_cust"],lmbda = _)
    train["ratings_given_by_cust"] = xt#(xt-xt.mean())/xt.std()
    test["ratings_given_by_cust"] = xte#(xte-xt.mean())/xt.std()
    
#     xt, _ = stats.boxcox(train["anon_var_1"])
#     xte = stats.boxcox(test["anon_var_1"],lmbda = _)
#     train["anon_var_1"] = xt#(xt-xt.mean())/xt.std()
#     test["anon_var_1"] = xte#(xte-xt.mean())/xt.std()

    
    xt, _ = stats.boxcox(train["anon_var_3"])
    xte = stats.boxcox(test["anon_var_3"],lmbda = _)
    train["anon_var_3"] = xt#(xt-xt.mean())/xt.std()
    test["anon_var_3"] = xte#(xte-xt.mean())/xt.std()

    return train, test

In [102]:
train_data = read_train()
train_data = hotenc(train_data)



Index(['id', 'distance', 'taxi_type', 'months_of_activity', 'customer_score',
       'customer_score_confidence', 'drop_location_type',
       'ratings_given_by_cust', 'num_of_cancelled_trips', 'anon_var_1',
       'anon_var_2', 'anon_var_3', 'sex', 'pricing_category'],
      dtype='object')
0        1
1        0
2        0
3        0
4        1
5        1
6        1
7        1
8        0
9        1
10       0
11       1
12       0
13       0
14       1
15       0
16       0
17       0
18       0
19       1
20       1
21       1
22       1
23       1
24       0
25       0
26       1
27       0
28       1
29       1
        ..
78967    0
78968    0
78969    1
78970    0
78971    0
78972    1
78973    0
78974    1
78975    1
78976    1
78977    1
78978    1
78979    0
78980    0
78981    1
78982    1
78983    1
78984    1
78985    1
78986    1
78987    1
78988    0
78989    0
78990    0
78991    1
78992    0
78993    1
78994    1
78995    1
78996    0
Name: anon_var_1, Length: 77673, dty

In [103]:
# test_data = pd.read_csv("test.csv")
# test_data = hotenc(test_data)
# train_data, test_data = imputer(train_data,test_data)
# train_data, test_data = boxCoxTransform(train_data,test_data)

In [104]:
data_train,data_test = train_test_split(train_data, test_size=0.2, shuffle = True)

data_train, data_test = imputer(data_train,data_test)
data_train, data_test = boxCoxTransform(data_train,data_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ver

In [105]:
print(data_test.shape)
print(data_train.columns)

(15535, 33)
Index(['id', 'distance', 'months_of_activity', 'ratings_given_by_cust',
       'num_of_cancelled_trips', 'anon_var_2', 'anon_var_3',
       'pricing_category', 'A', 'B', 'C', 'D', 'E',
       'A_Customerscore_confidence', 'B_Customerscore_confidence',
       'C_Customerscore_confidence', 'A_drop_location_type',
       'B_drop_location_type', 'C_drop_location_type', 'D_drop_location_type',
       'E_drop_location_type', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
       'Female', 'Male', 'anon_var_1anon_var_1_presence'],
      dtype='object')


In [106]:
def svm(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    
    clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)

In [ ]:
svm(data_train,data_test)

In [107]:
from sklearn.naive_bayes import BernoulliNB
def BernoulliNb(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = BernoulliNB()
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    outcome2 = clf.predict(xtrain)
    
    
    accuracy = accuracy_score(ytest,outcome)
    accuracy2 = accuracy_score(ytrain,outcome2)
    print(accuracy,accuracy2)    

In [108]:
BernoulliNb(data_train,data_test)

0.681943997425169 0.6833982426212624


In [109]:
from sklearn.naive_bayes import MultinomialNB
def Multinomial(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = MultinomialNB()
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    outcome2 = clf.predict(xtrain)
    
    
    accuracy = accuracy_score(ytest,outcome)
    accuracy2 = accuracy_score(ytrain,outcome2)
    print(accuracy,accuracy2)


In [110]:
Multinomial(data_train,data_test)

ValueError: Input X must be non-negative

In [111]:
from sklearn.ensemble import ExtraTreesClassifier
def ExtraTreeClassy(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = ExtraTreesClassifier(n_estimators = 500,n_jobs = -1, min_impurity_decrease=0.0077)
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    outcome2 = clf.predict(xtrain)
    
    
    accuracy = accuracy_score(ytest,outcome)
    accuracy2 = accuracy_score(ytrain,outcome2)
    print(accuracy,accuracy2)

In [112]:
ExtraTreeClassy(data_train,data_test)

0.6740263920180238 0.6720847146673533


In [113]:
from sklearn.naive_bayes import ComplementNB
def CompNb(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = ComplementNB()
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    outcome2 = clf.predict(xtrain)
    
    
    accuracy = accuracy_score(ytest,outcome)
    accuracy2 = accuracy_score(ytrain,outcome2)
    print(accuracy,accuracy2)


In [114]:
CompNb(data_train,data_test)

ValueError: Input X must be non-negative

In [115]:
from sklearn.linear_model import LogisticRegression
def LogReg(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = LogisticRegression(solver = "saga", max_iter = 1000,multi_class = 'multinomial',n_jobs = -1)
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    outcome = clf.predict(xtest)
    outcome2 = clf.predict(xtrain)
    
    
    accuracy = accuracy_score(ytest,outcome)
    accuracy2 = accuracy_score(ytrain,outcome2)
    print(accuracy,accuracy2)

In [82]:
LogReg(data_train,data_test)

0.6934663662697136 0.6950014483890695


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [116]:
from sklearn.linear_model import LogisticRegressionCV
def LogRegcv(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = LogisticRegressionCV(solver = "newton-cg", max_iter = 200,multi_class = 'multinomial',n_jobs = -1)
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)

In [84]:
LogRegcv(data_train,data_test)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.6943031863533956


In [117]:
from sklearn.neural_network import MLPClassifier
def MLP(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = MLPClassifier(activation = "logistic",solver = "sgd",learning_rate = 'adaptive', max_iter = 400)
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)

In [ ]:
MLP(data_train,data_test)

In [ ]:
from sklearn.neighbors import NearestCentroid
def NearCentroid(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = NearestCentroid()
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)


In [ ]:
NearCentroid(data_train,data_test)

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier
def RadNeighbour(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = RadiusNeighborsClassifier()
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)

In [ ]:
RadNeighbour(data_train,data_test)

In [ ]:
from sklearn.linear_model import RidgeClassifier
def Ridgereg(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = RidgeClassifier(alpha = 1,normalize=True,solver = 'sag')
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)

In [ ]:
Ridgereg(data_train,data_test)

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
def RidgeregCV(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = RidgeClassifierCV(normalize=True)
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)

In [ ]:
RidgeregCV(data_train,data_test)

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
def QuadraticDiscriminantAna(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = QuadraticDiscriminantAnalysis()
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)

In [ ]:
QuadraticDiscriminantAna(data_train,data_test)

In [ ]:
from sklearn.linear_model import SGDClassifier
def SGDClassy(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = SGDClassifier(loss='modified_huber',penalty='elasticnet',alpha = 0.001,n_jobs = -1,fit_intercept=True)
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)

In [ ]:
SGDClassy(data_train,data_test)

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
def PassiveAggressiveClass(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    
    clf = PassiveAggressiveClassifier(n_iter = 200,fit_intercept=True,tol = 0.0001,validation_fraction = 0.77,n_jobs = -1)
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)

In [ ]:
PassiveAggressiveClass(data_train,data_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def RandomForestClassi(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
#    
    clf = RandomForestClassifier( n_estimators = 500, criterion = 'gini',max_depth = 8, n_jobs = -1,oob_score = True,bootstrap = 0.77,max_features='sqrt')
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)

In [ ]:
RandomForestClassi(data_train,data_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# def Voting(training_data,testing_data):
training_data = data_train.copy()
testing_data = data_test.copy()


ytrain = training_data["pricing_category"]
xtrain = training_data.drop("pricing_category",axis = 1)
xtrain = xtrain.drop("id",axis = 1)

ytest = testing_data["pricing_category"]
xtest = testing_data.drop("pricing_category",axis = 1)
xtest = xtest.drop("id",axis = 1)



clf1 = BernoulliNB()
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)


#clf2 = MultinomialNB()
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)

# clf2 = ExtraTreesClassifier(n_estimators = 200,n_jobs = -1)
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)


clf3 = ComplementNB()
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)


clf4 = LogisticRegression(solver = "saga", max_iter = 1000,multi_class = 'multinomial',n_jobs = -1)
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)



# clf5 = LogisticRegressionCV(solver = "newton-cg", max_iter = 200,multi_class = 'multinomial',n_jobs = -1)
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)



clf6 = MLPClassifier(activation = "logistic",solver = "sgd",learning_rate = 'adaptive', max_iter = 400)
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)



clf7 = RidgeClassifier(alpha = 1,normalize=True,solver = 'sag')
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)



# clf8 = RidgeClassifierCV(normalize=True)
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)




# clf9 = QuadraticDiscriminantAnalysis()
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)


clf10 = SGDClassifier(loss='modified_huber',penalty='elasticnet',alpha = 0.001,n_jobs = -1,fit_intercept=True)
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)
#     print(accuracy)


clf11 = PassiveAggressiveClassifier(n_iter = 200,fit_intercept=True,tol = 0.0001,validation_fraction = 0.77,n_jobs = -1)
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)
#     print(accuracy)




clf12 = RandomForestClassifier( n_estimators = 500, criterion = 'gini',max_depth = 8, n_jobs = -1,oob_score = True,bootstrap = 0.77,max_features='sqrt')
#     clf.fit(xtrain, ytrain)
#     outcome = clf.predict(xtest)
#     accuracy = accuracy_score(ytest,outcome)




clf13 = XGBClassifier(nthread=-1,max_depth = 6, learning_rate=0.077,n_estimators=350,subsample=0.77,colsample_bynode=0.75)



eclf = VotingClassifier(estimators=[('1', clf1), ('12',clf12),('13',clf13)],voting='soft',n_jobs=-1)
# ('10',clf10)
# ('3', clf3), ('2', clf2),(5',clf5),('8', clf8)('9', clf9),#,('6',clf6),('7', clf7),('7', clf7), ('11',clf11) ,  ('10',clf10),('11',clf11),('12',clf12),('13',clf13)], 
eclf.fit(xtrain, ytrain)

outcome = eclf.predict(xtest)
accuracy = accuracy_score(ytest,outcome)
print(accuracy)

In [ ]:
Voting(data_train,data_test)

###### Bayesian Optimizations

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error

In [9]:
training_data = data_train.copy()
testing_data = data_test.copy()

ytrain = training_data["pricing_category"]
xtrain = training_data.drop("pricing_category",axis = 1)
xtrain = xtrain.drop("id",axis = 1)

ytest = testing_data["pricing_category"]
xtest = testing_data.drop("pricing_category",axis = 1)
xtest = xtest.drop("id",axis = 1)



dtrain = xgb.DMatrix(xtrain, label=ytrain)

/home/udit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/udit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [47]:
def xgb_evaluate(learning_rate,max_depth, gamma, colsample_bytree,n_estimators):
    params = {'learning_rate': learning_rate,'max_depth': int(max_depth),'subsample': 0.8,'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree,
             'n_estimators' : n_estimators,
             'objective':reg:logistic}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=400, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

SyntaxError: invalid syntax (<ipython-input-47-e69a9057ffca>, line 6)

In [48]:
xgb_bo = BayesianOptimization(xgb_evaluate, {'learning_rate': (0.01, 1.0),'max_depth': (3, 7), 'gamma': (0, 1),'colsample_bytree': (0.3, 0.9),'n_estimators': (200, 500)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=3, n_iter=10, acq='ei')

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------------------


XGBoostError: b'[16:16:32] src/objective/regression_obj.cu:101: label must be in [0,1] for logistic regression\n\nStack trace returned 10 entries:\n[bt] (0) /home/udit/anaconda3/lib/libxgboost.so(dmlc::StackTrace[abi:cxx11](unsigned long)+0x27a) [0x7fdb0f9b487a]\n[bt] (1) /home/udit/anaconda3/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x40) [0x7fdb0f9b57e0]\n[bt] (2) /home/udit/anaconda3/lib/libxgboost.so(xgboost::obj::RegLossObj<xgboost::obj::LogisticRegression>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*)+0x573) [0x7fdb0fa6f5b3]\n[bt] (3) /home/udit/anaconda3/lib/libxgboost.so(xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*)+0x1f9) [0x7fdb0f9c9cc9]\n[bt] (4) /home/udit/anaconda3/lib/libxgboost.so(XGBoosterUpdateOneIter+0x48) [0x7fdb0fb4d228]\n[bt] (5) /home/udit/anaconda3/lib/python3.7/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7fd

In [37]:
print(np.array(xgb_bo.res))

[{'target': -0.5976786666666666, 'params': {'colsample_bytree': 0.6942893223805829, 'gamma': 0.06070609983581221, 'learning_rate': 0.4153263867311826, 'max_depth': 6.98651903146903, 'n_estimators': 307.75442499046574}}
 {'target': -0.5893056666666666, 'params': {'colsample_bytree': 0.5210342064222988, 'gamma': 0.7437199296558867, 'learning_rate': 0.6845914451567847, 'max_depth': 4.3916800036527075, 'n_estimators': 371.25962116207006}}
 {'target': -0.563813, 'params': {'colsample_bytree': 0.36942119269796003, 'gamma': 0.06792204242706901, 'learning_rate': 0.0447667829037498, 'max_depth': 6.605016240503023, 'n_estimators': 443.1736203745203}}
 {'target': -0.6200313333333333, 'params': {'colsample_bytree': 0.9, 'gamma': 0.0, 'learning_rate': 0.01, 'max_depth': 7.0, 'n_estimators': 500.0}}
 {'target': -0.6526473333333334, 'params': {'colsample_bytree': 0.3, 'gamma': 0.0, 'learning_rate': 0.01, 'max_depth': 7.0, 'n_estimators': 200.0}}
 {'target': -0.70892, 'params': {'colsample_bytree': 0.

In [38]:
print(xtest.columns)
print(xtrain.columns)

Index(['distance', 'months_of_activity', 'customer_score',
       'ratings_given_by_cust', 'num_of_cancelled_trips', 'anon_var_1',
       'anon_var_2', 'anon_var_3', 'A', 'B', 'C', 'D', 'E',
       'A_Customerscore_confidence', 'B_Customerscore_confidence',
       'C_Customerscore_confidence', 'A_drop_location_type',
       'B_drop_location_type', 'C_drop_location_type', 'D_drop_location_type',
       'E_drop_location_type', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
       'Female', 'Male'],
      dtype='object')
Index(['distance', 'months_of_activity', 'customer_score',
       'ratings_given_by_cust', 'num_of_cancelled_trips', 'anon_var_1',
       'anon_var_2', 'anon_var_3', 'A', 'B', 'C', 'D', 'E',
       'A_Customerscore_confidence', 'B_Customerscore_confidence',
       'C_Customerscore_confidence', 'A_drop_location_type',
       'B_drop_location_type', 'C_drop_location_type', 'D_drop_location_type',
       'E_drop_location_type', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
    

In [39]:
params = xgb_bo.res[2]['params']
params['max_depth'] = int(params['max_depth'])

In [40]:
model2 = xgb.train(params, dtrain, num_boost_round=400)


tets = xgb.DMatrix(xtest)

# Predict on testing and training set
y_pred = model2.predict(tets)
# y_train_pred = model2.predict(xtrain)

# Report testing and training RMSE
print(np.sqrt(mean_squared_error(ytest, y_pred)))
# print(np.sqrt(mean_squared_error(ytrain, y_train_pred)))

0.5769142530676593


In [118]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
def AdaBoostClass(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
#  

    clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=600,learning_rate=1,algorithm="SAMME")
#     clf = RandomForestClassifier( n_estimators = 500, criterion = 'gini',max_depth = 8, n_jobs = -1,oob_score = True,bootstrap = 0.77,max_features='sqrt')
#     clf = OneVsRestClassifier(SVC(kernel = "poly"),n_jobs =-1)
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    outcome2 = clf.predict(xtrain)
    
    
    accuracy = accuracy_score(ytest,outcome)
    accuracy2 = accuracy_score(ytrain,outcome2)
    print(accuracy,accuracy2)
    
    
    
#     bdt_real = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),n_estimators=600,learning_rate=1)


In [119]:
AdaBoostClass(data_train,data_test)

0.6930157708400386 0.6938266439215939


In [122]:
def xg(training_data,testing_data):
    ytrain = training_data["pricing_category"]
    xtrain = training_data.drop("pricing_category",axis = 1)
    xtrain = xtrain.drop("id",axis = 1)
    
    ytest = testing_data["pricing_category"]
    xtest = testing_data.drop("pricing_category",axis = 1)
    xtest = xtest.drop("id",axis = 1)
    clf = XGBClassifier(nthread=-1,max_depth = 4, learning_rate=0.01,n_estimators=5000,subsample=0.77,colsample_bynode=0.75,verbosity = 0)
    
    clf.fit(xtrain, ytrain)

    #     polynomial_svm = SVC(C = c, kernel = "poly",degree = deg,coef0 = 1, gamma = 1)

    #     polynomial_svm.fit(x,y)
    
    outcome = clf.predict(xtest)
    
    

    accuracy = accuracy_score(ytest,outcome)
    print(accuracy)

In [123]:
xg(data_train,data_test)

0.6996459607338268
